In [12]:
import sys
sys.path.append('../main')
import sql
import pandas as pd
from functools import reduce
def get(q):
    return pd.DataFrame(sql.execute(q))

In [13]:
q='''select count(*) as c from alert
where language in ('C','C++')
and is_invalid=0;'''
total = sql.execute(q)[0]['c']
total


60134

In [14]:
q='''select CWE, count(*) as prevalence from alert a
        join memory_error me on a.alert_type_id = me.alert_type_id
        where language in ('C','C++')
        and is_invalid=0
        and memory=1
        and CWE is not null
        group by CWE;'''
df=get(q)
df['rate (%)']=round((df.prevalence/total)*100,2)
df = df.sort_values('rate (%)',ascending=False)
df

,CWE,prevalence,rate (%)
0,476,8689,14.45
8,404,4250,7.07
1,119,2125,3.53
6,457,1862,3.10
5,190,1003,1.67
14,125,939,1.56
9,416,877,1.46
2,120,751,1.25
10,170,378,0.63
16,590,363,0.60


In [15]:
q='''select CWE, count(*) as fixed from alert a
join memory_error me on a.alert_type_id = me.alert_type_id
where language in ('C','C++')
and is_invalid=0
and memory=1
  and status='Fixed'
and CWE is not null
group by CWE;'''
temp= get(q)
df=reduce(lambda x,y:pd.merge(x,y,on='CWE',how='left'),[df,temp])
df.fillna(0, inplace=True)
df.fixed=round(df.fixed/df.prevalence*100,2)
df

,CWE,prevalence,rate (%),fixed
0,476,8689,14.45,81.86
1,404,4250,7.07,66.92
2,119,2125,3.53,67.06
3,457,1862,3.10,74.01
4,190,1003,1.67,54.24
5,125,939,1.56,67.41
6,416,877,1.46,73.43
7,120,751,1.25,78.70
8,170,378,0.63,68.78
9,590,363,0.60,50.96


In [16]:
q='''select CWE, count(*) as triaged_bug from alert a
join memory_error me on a.alert_type_id = me.alert_type_id
where language in ('C','C++')
and is_invalid=0
and memory=1
  and classification = 'Bug'
and CWE is not null
group by CWE;'''
temp= get(q)
df=reduce(lambda x,y:pd.merge(x,y,on='CWE',how='left'),[df,temp])
df.fillna(0, inplace=True)
df['triaged_bug']=round(df['triaged_bug']/df.prevalence*100,2)
df

,CWE,prevalence,rate (%),fixed,triaged_bug
0,476,8689,14.45,81.86,21.99
1,404,4250,7.07,66.92,14.33
2,119,2125,3.53,67.06,4.42
3,457,1862,3.10,74.01,15.52
4,190,1003,1.67,54.24,7.88
5,125,939,1.56,67.41,8.20
6,416,877,1.46,73.43,7.41
7,120,751,1.25,78.70,2.13
8,170,378,0.63,68.78,7.94
9,590,363,0.60,50.96,0.00


In [17]:
q='''select CWE, count(*) as triaged_fp from alert a
join memory_error me on a.alert_type_id = me.alert_type_id
where language in ('C','C++')
and is_invalid=0
and memory=1
  and (classification = 'False Positive' or classification='Intentional')
and CWE is not null
group by CWE;'''
temp= get(q)
df=reduce(lambda x,y:pd.merge(x,y,on='CWE',how='left'),[df,temp])
df.fillna(0, inplace=True)
df['triaged_fp']=round(df['triaged_fp']/df.prevalence*100,2)
df

,CWE,prevalence,rate (%),fixed,triaged_bug,triaged_fp
0,476,8689,14.45,81.86,21.99,4.12
1,404,4250,7.07,66.92,14.33,18.99
2,119,2125,3.53,67.06,4.42,12.89
3,457,1862,3.10,74.01,15.52,6.12
4,190,1003,1.67,54.24,7.88,7.48
5,125,939,1.56,67.41,8.20,11.82
6,416,877,1.46,73.43,7.41,19.27
7,120,751,1.25,78.70,2.13,3.60
8,170,378,0.63,68.78,7.94,9.26
9,590,363,0.60,50.96,0.00,6.89


In [18]:
q='''select CWE, count(*) as actionable from alert a
join memory_error me on a.alert_type_id = me.alert_type_id
join actionability a2 on a.id = a2.alert_id
where language in ('C','C++')
and is_invalid=0
and memory=1
and CWE is not null
and actionability=1
group by CWE;'''
temp= get(q)
df=reduce(lambda x,y:pd.merge(x,y,on='CWE',how='left'),[df,temp])
df.fillna(0, inplace=True)
df.actionable=round(df.actionable/df.prevalence*100,2)
df

,CWE,prevalence,rate (%),fixed,triaged_bug,triaged_fp,actionable
0,476,8689,14.45,81.86,21.99,4.12,58.42
1,404,4250,7.07,66.92,14.33,18.99,43.41
2,119,2125,3.53,67.06,4.42,12.89,33.22
3,457,1862,3.10,74.01,15.52,6.12,53.49
4,190,1003,1.67,54.24,7.88,7.48,35.39
5,125,939,1.56,67.41,8.20,11.82,42.81
6,416,877,1.46,73.43,7.41,19.27,36.26
7,120,751,1.25,78.70,2.13,3.60,15.98
8,170,378,0.63,68.78,7.94,9.26,56.88
9,590,363,0.60,50.96,0.00,6.89,30.03


In [19]:
q='''select CWE, lifespan from alert a
join memory_error me on a.alert_type_id = me.alert_type_id
join actionability a2 on a.id = a2.alert_id
join
(select a.id, datediff(s.date,first_detected) as lifespan from alert a
join snapshot s
on a.last_snapshot_id=s.id) t1
on a.id=t1.id
where language in ('C','C++')
and is_invalid=0
and memory=1
and CWE is not null
and actionability=1;'''
l=get(q)
l=l.groupby('CWE')[['lifespan']].median()
l

,lifespan
CWE,
119,278.0
120,515.0
125,244.0
129,244.0
131,35.0
170,379.5
188,454.0
190,315.0
197,1260.0


In [20]:
df=reduce(lambda x,y:pd.merge(x,y,on='CWE',how='left'),[df,l])
df.fillna(0, inplace=True)
df = df.rename(columns={'prevalence':'alert count', 'rate (%)':'count (%) out of total','fixed':'eliminated (%)', 'triaged_bug':'triaged_bug (%)',
                        'triaged_fp':'triaged false positive (%)',
                        'actionable':'actionable (%)', 'lifespan':'lifespan (days)'})
df

,CWE,alert count,count (%) out of total,eliminated (%),triaged_bug (%),triaged false positive (%),actionable (%),lifespan (days)
0,476,8689,14.45,81.86,21.99,4.12,58.42,44.0
1,404,4250,7.07,66.92,14.33,18.99,43.41,122.0
2,119,2125,3.53,67.06,4.42,12.89,33.22,278.0
3,457,1862,3.10,74.01,15.52,6.12,53.49,135.0
4,190,1003,1.67,54.24,7.88,7.48,35.39,315.0
5,125,939,1.56,67.41,8.20,11.82,42.81,244.0
6,416,877,1.46,73.43,7.41,19.27,36.26,71.0
7,120,751,1.25,78.70,2.13,3.60,15.98,515.0
8,170,378,0.63,68.78,7.94,9.26,56.88,379.5
9,590,363,0.60,50.96,0.00,6.89,30.03,322.0


In [21]:
len(df)

27

In [22]:
df.to_csv ('cwe_insights_on_coverity_data.csv', index = False, header=True)